In [26]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:

%autoreload
from DbConnector import DbConnector
from decouple import config
from upload_data import get_all_user_data, get_trackpoints_as_df, get_labels_as_df
import os
from json import loads

In [8]:
connection = DbConnector(PASSWORD=config('USERPWD'))
client = connection.client
db = connection.db

You are connected to the database: test_db
-----------------------------------------------



In [30]:
users = get_all_user_data('dataset/dataset/Data')

1. import all user ones
2. per activity
   1. upload all trackpoints with user id and random id
   2. upload the activity with references to trackpoint

In [37]:
dataset_base_path = 'dataset/dataset'

In [48]:
def create_coll(collection_name):
    collection = db.create_collection(collection_name)    
    print('Created collection: ', collection)

def drop_coll(collection_name):
    collection = db[collection_name]
    collection.drop()

In [69]:
def get_all_user_data(data_path):
    # create for each user entry in list to insert them all in once
    all_users = os.listdir(data_path)
    # transform it in tuple with (id, has_labels) set has_labels to False for all
    all_users = [{
        '_id': u,
        'has_labels': False
    } for u in all_users]
    with open(os.path.join(dataset_base_path, 'labeled_ids.txt')) as f:
        for line in f.readlines():
            # update all entries with idx in file to has_labels = True
            all_users[int(line.strip())]['has_labels'] = True
    
    return all_users

In [71]:
def upload_batch(json_list, collection_name):
    collection = db[collection_name]
    return collection.insert_many(json_list).inserted_ids

In [64]:
def upload_single_activity(activity_json):
    collection = db['Activity']
    collection.insert_one(activity_json)

In [74]:
drop_coll(collection_name='User')
drop_coll(collection_name='Activity')
drop_coll(collection_name='TrackPoint')
create_coll(collection_name='User')
create_coll(collection_name='Activity')
create_coll(collection_name='TrackPoint')

all_users = get_all_user_data(os.path.join(dataset_base_path, 'Data'))
upload_batch(collection_name='User', json_list=all_users)

current_activity_id = 0
for user in os.listdir(os.path.join(dataset_base_path, 'Data')):
    print("User", user)
    base_path = os.path.join(dataset_base_path, 'Data', user)
    labels_exists = os.path.exists(os.path.join(base_path, 'labels.txt'))
    if labels_exists:
        df_labels = get_labels_as_df(os.path.join(base_path, 'labels.txt'))
    for activity in os.listdir(os.path.join(base_path, 'Trajectory')):
        path_activity = os.path.join(base_path,'Trajectory',activity)
        df_trackpoints = get_trackpoints_as_df(path_activity)
        if activity_df.shape[0] <= 2500:
            start_time = df_trackpoints.head(1).values[0][0]
            end_time = df_trackpoints.tail(1).values[0][0]
            transportation_label = ''
            if labels_exists:
                matching_row = df_labels.loc[(df_labels['Start Time'] == start_time) & ((df_labels['End Time']) == end_time)]
                if matching_row.shape[0] > 0:
                    transportation_label = matching_row.values[0][2]
            
            
            df_trackpoints.drop(columns=['date_time'], inplace=True)
            df_trackpoints['user_id'] = user
            df_trackpoints = df_trackpoints.reindex(columns=['user_id', 'Latitude', 'Longitude', 'Altitude', 'date_days', 'Date'])
            inserted_batch_ids = upload_batch(collection_name='Activity', json_list=loads(df_trackpoints.to_json(orient="records", date_format='iso')))

            activity_json = {
                'user_id': user,
                'transportation_label': transportation_label,
                'start_time': start_time.strftime('%Y-%m-%d %H:%M:%S'),
                'end_time': end_time.strftime('%Y-%m-%d %H:%M:%S'),
                'trackpoints': inserted_batch_ids
            }
            upload_single_activity(activity_json)
            current_activity_id += 1

Created collection:  Collection(Database(MongoClient(host=['tdt4225-29.idi.ntnu.no:27017'], document_class=dict, tz_aware=False, connect=True), 'test_db'), 'User')
Created collection:  Collection(Database(MongoClient(host=['tdt4225-29.idi.ntnu.no:27017'], document_class=dict, tz_aware=False, connect=True), 'test_db'), 'Activity')
Created collection:  Collection(Database(MongoClient(host=['tdt4225-29.idi.ntnu.no:27017'], document_class=dict, tz_aware=False, connect=True), 'test_db'), 'TrackPoint')
User 000
User 001
User 002
User 003
User 004
User 005
User 006
User 007
User 008
User 009
User 010


KeyboardInterrupt: 